In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import plotly.express as px

sns.set_theme(style="darkgrid")
current_palette=sns.color_palette()

### Exploratory Data Analysis
Data is analysed by month, quarter and year from 2018 t0 2022

- Load Data
- Types
- Mutation
- Apply
- Aggregation

#### Data Preparation [cleaning and processing]
| Action                                  | Description                                                                                                                                                                          |
|-----------------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Renaming columns                        | ----                                                                                                                                                                                 |
| Analysing column types for memory usage | Get an insight of datatypes to analyse memory usage e.g. float64 might be an indication of missing values, int64 tells me that there are no missing values, object is not super fast |
| Formatting data                         | Getting the right types to enable correct analysis e.g. converting string date to date type                                                                                          |
| Identifying duplicated columns          | ----                                                                                                                                                                                 |
| Fixing missing values                   | ML algorithms will chock if they encounter missing values                                                                                                                            |
| Dropping irrelevant columns             | ------                                                                                                                                                                               |
| Feature selection                       | -------                                                                                                                                                                              |
| Data visualisations                     | ------                                                                                                                                                                               |




----------------------------------------------------------------------------------------------


In [2]:
pd.options.display.min_rows = 20
pd.options.display.max_columns = 50

In [3]:
## Read in January 2022 data from filesystem for 1st analysis
df_flights_012022 = pd.read_csv("../../data/eda/flights/flight_data_2022/flight_data_202201.zip", low_memory=False)

In [4]:
df_flights_012022.shape

(563737, 120)

In [5]:
df_flights_012022.head(2)

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Originally_Scheduled_Code_Share_Airline,DOT_ID_Originally_Scheduled_Code_Share_Airline,IATA_Code_Originally_Scheduled_Code_Share_Airline,Flight_Num_Originally_Scheduled_Code_Share_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,...,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum,Duplicate,Unnamed: 119
0,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,1581,NaN,NaN,NaN,NaN,DL,19790,DL,N315DN,1581,11697,1169706,32467,FLL,"Fort Lauderdale, FL",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN
1,2022,1,1,6,4,2022-01-06,DL,DL,19790,DL,1582,NaN,NaN,NaN,NaN,DL,19790,DL,N545US,1582,10397,1039707,30397,ATL,"Atlanta, GA",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN


In [6]:
df_flights_012022.columns.to_list()

['Year',
 'Quarter',
 'Month',
 'DayofMonth',
 'DayOfWeek',
 'FlightDate',
 'Marketing_Airline_Network',
 'Operated_or_Branded_Code_Share_Partners',
 'DOT_ID_Marketing_Airline',
 'IATA_Code_Marketing_Airline',
 'Flight_Number_Marketing_Airline',
 'Originally_Scheduled_Code_Share_Airline',
 'DOT_ID_Originally_Scheduled_Code_Share_Airline',
 'IATA_Code_Originally_Scheduled_Code_Share_Airline',
 'Flight_Num_Originally_Scheduled_Code_Share_Airline',
 'Operating_Airline ',
 'DOT_ID_Operating_Airline',
 'IATA_Code_Operating_Airline',
 'Tail_Number',
 'Flight_Number_Operating_Airline',
 'OriginAirportID',
 'OriginAirportSeqID',
 'OriginCityMarketID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'OriginStateFips',
 'OriginStateName',
 'OriginWac',
 'DestAirportID',
 'DestAirportSeqID',
 'DestCityMarketID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateFips',
 'DestStateName',
 'DestWac',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 '

In [7]:
# remove spaces from column names
df_flights_012022= df_flights_012022.rename(columns={'Operating_Airline ' : 'Operating_Airline'})

In [8]:
df_flights_012022["Operating_Airline"].head(1)

0    DL
Name: Operating_Airline, dtype: object

Column Selction

In [9]:
# Drop irrelevant columns
SELECTED_COLUMNS = [
    # 'Year',
    # 'Quarter',
    # 'Month',
    # 'DayofMonth',
    # 'DayOfWeek',
    'FlightDate',
    # 'Marketing_Airline_Network',
    # 'Operated_or_Branded_Code_Share_Partners',
    # 'DOT_ID_Marketing_Airline',
    # 'IATA_Code_Marketing_Airline',
    # 'Flight_Number_Marketing_Airline',
    # 'Originally_Scheduled_Code_Share_Airline',
    # 'DOT_ID_Originally_Scheduled_Code_Share_Airline',
    # 'IATA_Code_Originally_Scheduled_Code_Share_Airline',
    # 'Flight_Num_Originally_Scheduled_Code_Share_Airline',
    'Operating_Airline',
    # 'DOT_ID_Operating_Airline',
    # 'IATA_Code_Operating_Airline',
    # 'Tail_Number',
    # 'Flight_Number_Operating_Airline',
    'OriginAirportID',
    # 'OriginAirportSeqID',
    # 'OriginCityMarketID',
    'Origin',
    'OriginCityName',
    'OriginState',
    # 'OriginStateFips',
    # 'OriginStateName',
    # 'OriginWac',
    'DestAirportID',
    # 'DestAirportSeqID',
    # 'DestCityMarketID',
    'Dest',
    'DestCityName',
    'DestState',
    # 'DestStateFips',
    'DestStateName',
    # 'DestWac',
    'CRSDepTime',
    'DepTime',
    'DepDelay',
    'DepDelayMinutes',
    'DepDel15',
    'DepartureDelayGroups',
    'DepTimeBlk',
    'TaxiOut',
    'WheelsOff',
    'WheelsOn',
    'TaxiIn',
    'CRSArrTime',
    'ArrTime',
    'ArrDelay',
    'ArrDelayMinutes',
    'ArrDel15',
    'ArrivalDelayGroups',
    'ArrTimeBlk',
    'Cancelled',
    'CancellationCode',
    'Diverted',
    'CRSElapsedTime',
    'ActualElapsedTime',
    'AirTime',
    'Flights',
    'Distance',
    'DistanceGroup',
    'CarrierDelay',
    'WeatherDelay',
    'NASDelay',
    'SecurityDelay',
    'LateAircraftDelay',
    'FirstDepTime',
    'TotalAddGTime',
    'LongestAddGTime',
    # 'DivAirportLandings',
    # 'DivReachedDest',
    # 'DivActualElapsedTime',
    # 'DivArrDelay',
    # 'DivDistance',
    # 'Div1Airport',
    # 'Div1AirportID',
    # 'Div1AirportSeqID',
    # 'Div1WheelsOn',
    # 'Div1TotalGTime',
    # 'Div1LongestGTime',
    # 'Div1WheelsOff',
    # 'Div1TailNum',
    # 'Div2Airport',
    # 'Div2AirportID',
    # 'Div2AirportSeqID',
    # 'Div2WheelsOn',
    # 'Div2TotalGTime',
    # 'Div2LongestGTime',
    # 'Div2WheelsOff',
    # 'Div2TailNum',
    # 'Div3Airport',
    # 'Div3AirportID',
    # 'Div3AirportSeqID',
    # 'Div3WheelsOn',
    # 'Div3TotalGTime',
    # 'Div3LongestGTime',
    # 'Div3WheelsOff',
    # 'Div3TailNum',
    # 'Div4Airport',
    # 'Div4AirportID',
    # 'Div4AirportSeqID',
    # 'Div4WheelsOn',
    # 'Div4TotalGTime',
    # 'Div4LongestGTime',
    # 'Div4WheelsOff',
    # 'Div4TailNum',
    # 'Div5Airport',
    # 'Div5AirportID',
    # 'Div5AirportSeqID',
    # 'Div5WheelsOn',
    # 'Div5TotalGTime',
    # 'Div5LongestGTime',
    # 'Div5WheelsOff',
    # 'Div5TailNum',
    # 'Duplicate',
    # 'Unnamed: 119'
]

In [10]:
## Read in Jan - Aug 2022 data from filesystem
df_flights_012022_082022 = pd.read_csv("../../data/eda/flights/flight_data_Y2022/flight_data_2022.csv.zip",  low_memory=False)



In [11]:
df_flights_012022_082022= df_flights_012022_082022.rename(columns={'Operating_Airline ' : 'Operating_Airline'})
df_flights_012022_082022.shape

(4691967, 121)

In [12]:
df_flights_012022_082022 = df_flights_012022_082022[SELECTED_COLUMNS].copy()
df_flights_012022_082022.columns.to_list()

['FlightDate',
 'Operating_Airline',
 'OriginAirportID',
 'Origin',
 'OriginCityName',
 'OriginState',
 'DestAirportID',
 'Dest',
 'DestCityName',
 'DestState',
 'DestStateName',
 'CRSDepTime',
 'DepTime',
 'DepDelay',
 'DepDelayMinutes',
 'DepDel15',
 'DepartureDelayGroups',
 'DepTimeBlk',
 'TaxiOut',
 'WheelsOff',
 'WheelsOn',
 'TaxiIn',
 'CRSArrTime',
 'ArrTime',
 'ArrDelay',
 'ArrDelayMinutes',
 'ArrDel15',
 'ArrivalDelayGroups',
 'ArrTimeBlk',
 'Cancelled',
 'CancellationCode',
 'Diverted',
 'CRSElapsedTime',
 'ActualElapsedTime',
 'AirTime',
 'Flights',
 'Distance',
 'DistanceGroup',
 'CarrierDelay',
 'WeatherDelay',
 'NASDelay',
 'SecurityDelay',
 'LateAircraftDelay',
 'FirstDepTime',
 'TotalAddGTime',
 'LongestAddGTime']

Types

In [13]:
## Types -> Getting the right types will enable analysis and correctness
df_flights_012022_082022[SELECTED_COLUMNS].dtypes

FlightDate               object
Operating_Airline        object
OriginAirportID           int64
Origin                   object
OriginCityName           object
OriginState              object
DestAirportID             int64
Dest                     object
DestCityName             object
DestState                object
DestStateName            object
CRSDepTime                int64
DepTime                 float64
DepDelay                float64
DepDelayMinutes         float64
DepDel15                float64
DepartureDelayGroups    float64
DepTimeBlk               object
TaxiOut                 float64
WheelsOff               float64
WheelsOn                float64
TaxiIn                  float64
CRSArrTime                int64
ArrTime                 float64
ArrDelay                float64
ArrDelayMinutes         float64
ArrDel15                float64
ArrivalDelayGroups      float64
ArrTimeBlk               object
Cancelled               float64
CancellationCode         object
Diverted

In [14]:
## Evaluate memory usage
df_flights_012022_082022[SELECTED_COLUMNS].memory_usage(deep=True)

Index                         128
FlightDate              314361789
Operating_Airline       276826053
OriginAirportID          37535736
Origin                  281518020
OriginCityName          328758959
OriginState             276826053
DestAirportID            37535736
Dest                    281518020
DestCityName            328758751
DestState               276826053
DestStateName           305896341
CRSDepTime               37535736
DepTime                  37535736
DepDelay                 37535736
DepDelayMinutes          37535736
DepDel15                 37535736
DepartureDelayGroups     37535736
DepTimeBlk              309669822
TaxiOut                  37535736
WheelsOff                37535736
WheelsOn                 37535736
TaxiIn                   37535736
CRSArrTime               37535736
ArrTime                  37535736
ArrDelay                 37535736
ArrDelayMinutes          37535736
ArrDel15                 37535736
ArrivalDelayGroups       37535736
ArrTimeBlk    

In [15]:
df_flights_012022_082022[SELECTED_COLUMNS].memory_usage(deep=True).sum()

4720593329

In [16]:
# More data cleaning 
## Format column data from [0,1] -> boolean
df_flights_012022_082022["Cancelled"] = df_flights_012022_082022["Cancelled"].astype("bool")
## Converting dataTypes
df_flights_012022_082022["FlightDate"] = pd.to_datetime(df_flights_012022_082022["FlightDate"])
df_flights_012022_082022["FlightDate"].head() 

0   2022-01-06
1   2022-01-06
2   2022-01-06
3   2022-01-06
4   2022-01-06
Name: FlightDate, dtype: datetime64[ns]

In [17]:
df_flights_012022_082022[SELECTED_COLUMNS].memory_usage(deep=True).sum()

4410923507

In [18]:
642379051 / 1724280395

0.37254906618595524

Integer Types

In [19]:
df_flights_012022_082022.head()

,FlightDate,Operating_Airline,OriginAirportID,Origin,OriginCityName,OriginState,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime
0,2022-01-06,DL,11697,FLL,"Fort Lauderdale, FL",FL,12953,LGA,"New York, NY",NY,New York,1126,NaN,NaN,NaN,NaN,NaN,1100-1159,NaN,NaN,NaN,NaN,1419,NaN,NaN,NaN,NaN,NaN,1400-1459,True,A,0.0,173.0,NaN,NaN,1.0,1076.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-06,DL,10397,ATL,"Atlanta, GA",GA,11697,FLL,"Fort Lauderdale, FL",FL,Florida,1631,1627.0,-4.0,0.0,0.0,-1.0,1600-1659,15.0,1642.0,1815.0,5.0,1821,1820.0,-1.0,0.0,0.0,-1.0,1800-1859,False,NaN,0.0,110.0,113.0,93.0,1.0,581.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-06,DL,11697,FLL,"Fort Lauderdale, FL",FL,10397,ATL,"Atlanta, GA",GA,Georgia,1931,1929.0,-2.0,0.0,0.0,-1.0,1900-1959,13.0,1942.0,2105.0,10.0,2127,2115.0,-12.0,0.0,0.0,-1.0,2100-2159,False,NaN,0.0,116.0,106.0,83.0,1.0,581.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-06,DL,11697,FLL,"Fort Lauderdale, FL",FL,14492,RDU,"Raleigh/Durham, NC",NC,North Carolina,1024,1019.0,-5.0,0.0,0.0,-1.0,1000-1059,17.0,1036.0,1209.0,3.0,1227,1212.0,-15.0,0.0,0.0,-1.0,1200-1259,False,NaN,0.0,123.0,113.0,93.0,1.0,680.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-06,DL,10397,ATL,"Atlanta, GA",GA,12448,JAN,"Jackson/Vicksburg, MS",MS,Mississippi,1117,1113.0,-4.0,0.0,0.0,-1.0,1100-1159,14.0,1127.0,1127.0,4.0,1142,1131.0,-11.0,0.0,0.0,-1.0,1100-1159,False,NaN,0.0,85.0,78.0,60.0,1.0,341.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
(df_flights_012022_082022[SELECTED_COLUMNS].select_dtypes(int).describe())


,OriginAirportID,DestAirportID,CRSDepTime,CRSArrTime,DistanceGroup
count,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06
mean,1.265936e+04,1.265932e+04,1.330279e+03,1.485272e+03,3.662583e+00
std,1.523274e+03,1.523281e+03,4.912575e+02,5.196737e+02,2.325003e+00
min,1.013500e+04,1.013500e+04,1.000000e+00,1.000000e+00,1.000000e+00
25%,1.129200e+04,1.129200e+04,9.130000e+02,1.101000e+03,2.000000e+00
50%,1.288900e+04,1.288900e+04,1.320000e+03,1.512000e+03,3.000000e+00
75%,1.402700e+04,1.402700e+04,1.735000e+03,1.920000e+03,5.000000e+00
max,1.686900e+04,1.686900e+04,2.359000e+03,2.400000e+03,1.100000e+01


In [21]:
(df_flights_012022_082022.shape)

(4691967, 46)

In [22]:
(df_flights_012022_082022.OriginAirportID.describe())

count    4.691967e+06
mean     1.265936e+04
std      1.523274e+03
min      1.013500e+04
25%      1.129200e+04
50%      1.288900e+04
75%      1.402700e+04
max      1.686900e+04
Name: OriginAirportID, dtype: float64

In [23]:
np.iinfo(np.int16)

iinfo(min=-32768, max=32767, dtype=int16)

In [24]:
np.iinfo(np.int64)

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

Floats

In [25]:
(df_flights_012022_082022[SELECTED_COLUMNS].select_dtypes('float'))

,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,173.0,NaN,NaN,1.0,1076.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1627.0,-4.0,0.0,0.0,-1.0,15.0,1642.0,1815.0,5.0,1820.0,-1.0,0.0,0.0,-1.0,0.0,110.0,113.0,93.0,1.0,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1929.0,-2.0,0.0,0.0,-1.0,13.0,1942.0,2105.0,10.0,2115.0,-12.0,0.0,0.0,-1.0,0.0,116.0,106.0,83.0,1.0,581.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1019.0,-5.0,0.0,0.0,-1.0,17.0,1036.0,1209.0,3.0,1212.0,-15.0,0.0,0.0,-1.0,0.0,123.0,113.0,93.0,1.0,680.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1113.0,-4.0,0.0,0.0,-1.0,14.0,1127.0,1127.0,4.0,1131.0,-11.0,0.0,0.0,-1.0,0.0,85.0,78.0,60.0,1.0,341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1230.0,-7.0,0.0,0.0,-1.0,11.0,1241.0,1430.0,5.0,1435.0,-23.0,0.0,0.0,-2.0,0.0,81.0,65.0,49.0,1.0,341.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,857.0,-3.0,0.0,0.0,-1.0,13.0,910.0,1030.0,8.0,1038.0,-12.0,0.0,0.0,-1.0,0.0,110.0,101.0,80.0,1.0,481.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1018.0,18.0,18.0,1.0,1.0,15.0,1033.0,1431.0,4.0,1435.0,2.0,2.0,0.0,0.0,0.0,213.0,197.0,178.0,1.0,1416.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1422.0,8.0,8.0,0.0,0.0,11.0,1433.0,1528.0,3.0,1531.0,-2.0,0.0,0.0,-1.0,0.0,79.0,69.0,55.0,1.0,356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1640.0,-3.0,0.0,0.0,-1.0,12.0,1652.0,1759.0,4.0,1803.0,-11.0,0.0,0.0,-1.0,0.0,91.0,83.0,67.0,1.0,356.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
(df_flights_012022_082022.DepTime.describe())


count    4.556687e+06
mean     1.334551e+03
std      5.068218e+02
min      1.000000e+00
25%      9.160000e+02
50%      1.324000e+03
75%      1.745000e+03
max      2.400000e+03
Name: DepTime, dtype: float64

In [27]:
#oops! 135280 missing values [NaN: 135280]
(df_flights_012022_082022.DepTime.value_counts(dropna=False))

NaN      135280
555.0     12168
556.0     11320
557.0     11308
558.0     10612
559.0     10081
655.0      9483
600.0      9451
554.0      9344
656.0      8862
          ...  
421.0         9
400.0         8
346.0         8
423.0         8
422.0         7
425.0         7
353.0         5
403.0         5
428.0         4
441.0         2
Name: DepTime, Length: 1441, dtype: int64

In [28]:
# investigate where these values are missing
(df_flights_012022_082022[SELECTED_COLUMNS].query('DepTime.isna()'))

,FlightDate,Operating_Airline,OriginAirportID,Origin,OriginCityName,OriginState,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime
0,2022-01-06,DL,11697,FLL,"Fort Lauderdale, FL",FL,12953,LGA,"New York, NY",NY,New York,1126,NaN,NaN,NaN,NaN,NaN,1100-1159,NaN,NaN,NaN,NaN,1419,NaN,NaN,NaN,NaN,NaN,1400-1459,True,A,0.0,173.0,NaN,NaN,1.0,1076.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,2022-01-06,DL,11433,DTW,"Detroit, MI",MI,10693,BNA,"Nashville, TN",TN,Tennessee,2120,NaN,NaN,NaN,NaN,NaN,2100-2159,NaN,NaN,NaN,NaN,2216,NaN,NaN,NaN,NaN,NaN,2200-2259,True,B,0.0,116.0,NaN,NaN,1.0,456.0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75,2022-01-06,DL,11193,CVG,"Cincinnati, OH",KY,10721,BOS,"Boston, MA",MA,Massachusetts,1700,NaN,NaN,NaN,NaN,NaN,1700-1759,NaN,NaN,NaN,NaN,1903,NaN,NaN,NaN,NaN,NaN,1900-1959,True,B,0.0,123.0,NaN,NaN,1.0,752.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
148,2022-01-06,DL,13303,MIA,"Miami, FL",FL,12953,LGA,"New York, NY",NY,New York,1958,NaN,NaN,NaN,NaN,NaN,1900-1959,NaN,NaN,NaN,NaN,2307,NaN,NaN,NaN,NaN,NaN,2300-2359,True,B,0.0,189.0,NaN,NaN,1.0,1096.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,2022-01-06,DL,10397,ATL,"Atlanta, GA",GA,10693,BNA,"Nashville, TN",TN,Tennessee,1640,NaN,NaN,NaN,NaN,NaN,1600-1659,NaN,NaN,NaN,NaN,1646,NaN,NaN,NaN,NaN,NaN,1600-1659,True,B,0.0,66.0,NaN,NaN,1.0,214.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158,2022-01-06,DL,10693,BNA,"Nashville, TN",TN,10397,ATL,"Atlanta, GA",GA,Georgia,1756,NaN,NaN,NaN,NaN,NaN,1700-1759,NaN,NaN,NaN,NaN,2010,NaN,NaN,NaN,NaN,NaN,2000-2059,True,B,0.0,74.0,NaN,NaN,1.0,214.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,2022-01-06,DL,11618,EWR,"Newark, NJ",NJ,10397,ATL,"Atlanta, GA",GA,Georgia,600,NaN,NaN,NaN,NaN,NaN,0600-0659,NaN,NaN,NaN,NaN,829,NaN,NaN,NaN,NaN,NaN,0800-0859,True,A,0.0,149.0,NaN,NaN,1.0,746.0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
242,2022-01-06,DL,15304,TPA,"Tampa, FL",FL,12953,LGA,"New York, NY",NY,New York,1815,NaN,NaN,NaN,NaN,NaN,1800-1859,NaN,NaN,NaN,NaN,2053,NaN,NaN,NaN,NaN,NaN,2000-2059,True,B,0.0,158.0,NaN,NaN,1.0,1010.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,2022-01-06,DL,14683,SAT,"San Antonio, TX",TX,11433,DTW,"Detroit, MI",MI,Michigan,531,NaN,NaN,NaN,NaN,NaN,0001-0559,NaN,NaN,NaN,NaN,930,NaN,NaN,NaN,NaN,NaN,0900-0959,True,A,0.0,179.0,NaN,NaN,1.0,1214.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
378,2022-01-06,DL,13303,MIA,"Miami, FL",FL,12478,JFK,"New York, NY",NY,New York,1959,NaN,NaN,NaN,NaN,NaN,1900-1959,NaN,NaN,NaN,NaN,2302,NaN,NaN,NaN,NaN,NaN,2300-2359,True,B,0.0,183.0,NaN,NaN,1.0,1089.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Subject Matter Expert info => Missing values correspond to cancelled flights
# decision -> replace with 0
df_flights_012022_082022=(df_flights_012022_082022
 [SELECTED_COLUMNS]
 .assign(DepTime=df_flights_012022_082022.DepTime.fillna(0),
         DepDelay=df_flights_012022_082022.DepDelay.fillna(0),
         DepDelayMinutes=df_flights_012022_082022.DepDelayMinutes.fillna(0),
         DepDel15=df_flights_012022_082022.DepDel15.fillna(0),
         DepartureDelayGroups=df_flights_012022_082022.DepartureDelayGroups.fillna(0),
         TaxiOut=df_flights_012022_082022.TaxiOut.fillna(0),
         WheelsOff=df_flights_012022_082022.WheelsOff.fillna(0),
         WheelsOn=df_flights_012022_082022.WheelsOn.fillna(0),
         TaxiIn=df_flights_012022_082022.TaxiIn.fillna(0),
         ArrTime=df_flights_012022_082022.ArrTime.fillna(0),
         ArrDelay=df_flights_012022_082022.ArrDelay.fillna(0),
         ArrDelayMinutes=df_flights_012022_082022.ArrDelayMinutes.fillna(0),
         ArrDel15=df_flights_012022_082022.ArrDel15.fillna(0),
         ArrivalDelayGroups=df_flights_012022_082022.ArrivalDelayGroups.fillna(0),
         Diverted=df_flights_012022_082022.Diverted.fillna(0),
         CRSElapsedTime=df_flights_012022_082022.CRSElapsedTime.fillna(0),
         ActualElapsedTime=df_flights_012022_082022.ActualElapsedTime.fillna(0),
         AirTime=df_flights_012022_082022.AirTime.fillna(0),
         Flights=df_flights_012022_082022.Flights.fillna(0),
         Distance=df_flights_012022_082022.Distance.fillna(0),
         CarrierDelay=df_flights_012022_082022.CarrierDelay.fillna(0),
         WeatherDelay=df_flights_012022_082022.WeatherDelay.fillna(0),
         NASDelay=df_flights_012022_082022.NASDelay.fillna(0),
         SecurityDelay=df_flights_012022_082022.SecurityDelay.fillna(0),
         LateAircraftDelay=df_flights_012022_082022.LateAircraftDelay.fillna(0),
         FirstDepTime=df_flights_012022_082022.FirstDepTime.fillna(0),
         TotalAddGTime=df_flights_012022_082022.TotalAddGTime.fillna(0),
         LongestAddGTime=df_flights_012022_082022.LongestAddGTime.fillna(0))
 )


In [30]:
(df_flights_012022_082022[SELECTED_COLUMNS].describe())

,OriginAirportID,DestAirportID,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime
count,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4691967.0,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06,4.691967e+06
mean,1.265936e+04,1.265932e+04,1.330279e+03,1.296072e+03,1.281724e+01,1.566092e+01,2.133174e-01,2.138723e-01,1.645683e+01,1.316564e+03,1.410041e+03,7.674164e+00,1.485272e+03,1.412782e+03,7.420364e+00,1.536775e+01,2.102536e-01,-5.353874e-02,2.577171e-03,1.410956e+02,1.313287e+02,1.072756e+02,1.0,7.977327e+02,3.662583e+00,5.680550e+00,8.143591e-01,2.445492e+00,3.104306e-02,5.391095e+00,8.801582e+00,2.718606e-01,2.660775e-01
std,1.523274e+03,1.523281e+03,4.912575e+02,5.471130e+02,5.287348e+01,5.190498e+01,4.096500e-01,2.320545e+00,9.790165e+00,5.513330e+02,5.860965e+02,6.729654e+00,5.196737e+02,5.910546e+02,5.460918e+01,5.148330e+01,4.074887e-01,2.454217e+00,5.070039e-02,7.181163e+01,7.466200e+01,7.151418e+01,0.0,5.925771e+02,2.325003e+00,3.619968e+01,1.412358e+01,1.472690e+01,1.837461e+00,2.729813e+01,1.161970e+02,4.566807e+00,4.424696e+00
min,1.013500e+04,1.013500e+04,1.000000e+00,0.000000e+00,-7.800000e+01,0.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,-1.000000e+02,0.000000e+00,0.000000e+00,-2.000000e+00,0.000000e+00,-4.800000e+01,0.000000e+00,0.000000e+00,1.0,3.100000e+01,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.129200e+04,1.129200e+04,9.130000e+02,8.540000e+02,-5.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,1.100000e+01,9.090000e+02,1.016000e+03,4.000000e+00,1.101000e+03,1.019000e+03,-1.400000e+01,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,8.900000e+01,8.000000e+01,5.700000e+01,1.0,3.660000e+02,2.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.288900e+04,1.288900e+04,1.320000e+03,1.309000e+03,-1.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,1.400000e+01,1.322000e+03,1.439000e+03,6.000000e+00,1.512000e+03,1.442000e+03,-5.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,0.000000e+00,1.240000e+02,1.160000e+02,9.100000e+01,1.0,6.420000e+02,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,1.402700e+04,1.402700e+04,1.735000e+03,1.737000e+03,1.000000e+01,1.000000e+01,0.000000e+00,0.000000e+00,1.900000e+01,1.751000e+03,1.902000e+03,9.000000e+00,1.920000e+03,1.906000e+03,1.000000e+01,1.000000e+01,0.000000e+00,0.000000e+00,0.000000e+00,1.710000e+02,1.650000e+02,1.390000e+02,1.0,1.034000e+03,5.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
max,1.686900e+04,1.686900e+04,2.359000e+03,2.400000e+03,7.223000e+03,7.223000e+03,1.000000e+00,1.200000e+01,2.210000e+02,2.400000e+03,2.400000e+03,2.900000e+02,2.400000e+03,2.400000e+03,7.232000e+03,7.232000e+03,1.000000e+00,1.200000e+01,1.000000e+00,6.900000e+02,7.640000e+02,7.270000e+02,1.0,5.095000e+03,1.100000e+01,7.232000e+03,2.363000e+03,1.740000e+03,1.245000e+03,2.098000e+03,2.400000e+03,3.760000e+02,3.410000e+02


In [31]:
(df_flights_012022_082022[SELECTED_COLUMNS].select_dtypes('float'))

,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,TaxiOut,WheelsOff,WheelsOn,TaxiIn,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,173.0,0.0,0.0,1.0,1076.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1627.0,-4.0,0.0,0.0,-1.0,15.0,1642.0,1815.0,5.0,1820.0,-1.0,0.0,0.0,-1.0,0.0,110.0,113.0,93.0,1.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1929.0,-2.0,0.0,0.0,-1.0,13.0,1942.0,2105.0,10.0,2115.0,-12.0,0.0,0.0,-1.0,0.0,116.0,106.0,83.0,1.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1019.0,-5.0,0.0,0.0,-1.0,17.0,1036.0,1209.0,3.0,1212.0,-15.0,0.0,0.0,-1.0,0.0,123.0,113.0,93.0,1.0,680.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1113.0,-4.0,0.0,0.0,-1.0,14.0,1127.0,1127.0,4.0,1131.0,-11.0,0.0,0.0,-1.0,0.0,85.0,78.0,60.0,1.0,341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1230.0,-7.0,0.0,0.0,-1.0,11.0,1241.0,1430.0,5.0,1435.0,-23.0,0.0,0.0,-2.0,0.0,81.0,65.0,49.0,1.0,341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,857.0,-3.0,0.0,0.0,-1.0,13.0,910.0,1030.0,8.0,1038.0,-12.0,0.0,0.0,-1.0,0.0,110.0,101.0,80.0,1.0,481.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,1018.0,18.0,18.0,1.0,1.0,15.0,1033.0,1431.0,4.0,1435.0,2.0,2.0,0.0,0.0,0.0,213.0,197.0,178.0,1.0,1416.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1422.0,8.0,8.0,0.0,0.0,11.0,1433.0,1528.0,3.0,1531.0,-2.0,0.0,0.0,-1.0,0.0,79.0,69.0,55.0,1.0,356.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1640.0,-3.0,0.0,0.0,-1.0,12.0,1652.0,1759.0,4.0,1803.0,-11.0,0.0,0.0,-1.0,0.0,91.0,83.0,67.0,1.0,356.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Delays by Airline in 2022

In [ ]:
airline_csv = "https://raw.githubusercontent.com/jenbam/airlines/master/data-raw/airlines.csv"
airlines = pd.read_csv(airline_csv)
## save locally
#airlines.to_csv("airlines.csv") 
airlines.query("Code == '9E'")
#airlines
airline_mapper = airlines.set_index("Code")["Description"]


In [ ]:
airline_counts=df_flights_012022_082022["Operating_Airline"].value_counts()
airline_counts.head()

In [ ]:
airline_counts=df_flights_012022_082022["Operating_Airline"].value_counts()
airline_counts
airline_counts.index=airline_counts.index.map(
    airlines.set_index('Code')['Description']
    ).rename("Airline")
airline_counts.head()

In [ ]:
airline_counts.plot(kind='barh', figsize=(10,10))

In [ ]:
airline_counts.sort_values(ascending=True).plot(kind='barh', figsize=(10,10))

In [ ]:
airline_counts.sort_values(ascending=True).to_frame("Row Count")

In [ ]:
px.bar(
    airline_counts.sort_values(ascending=True), 
    orientation='h', 
    template="plotly_dark",
    title="Flights per airline",
    height=700,
    )

In [ ]:
pct_cancelled = df_flights_012022_082022["Cancelled"].mean()
print(f"{pct_cancelled:0.2f}% of flights are cancelled")
pct_cancelled

In [ ]:
df_flights_012022_082022["Cancelled"].value_counts()

### Cancelled flights, group by Year

In [ ]:
#df_flights.query("Year==2022").query("Cancelled")
df_flights_012022_082022.groupby([df_flights_012022_082022["FlightDate"].dt.year])["Cancelled"].agg("mean").to_frame()

In [ ]:
#df_flights.query("Year==2022").query("Cancelled")
df_flights_012022_082022.groupby([df_flights_012022_082022["FlightDate"].dt.year])["Cancelled"].agg("mean").to_frame()

In [ ]:
df_flights_012022_082022.query('20220101 < FlightDate < 20221231')  # flights in 2022

In [ ]:
cancelled_flights_analysis = (
    (df_flights_012022_082022.query("20220101 <FlightDate < 20221231")
     .groupby(["Operating_Airline", "Cancelled"])['Cancelled']
     .size()
     .unstack()
     .reset_index()
     )
    .rename(columns={False: "Departed", True: "Cancelled"})
    .rename_axis(None, axis=1))

cancelled_flights_analysis["Total"] = cancelled_flights_analysis["Departed"] + cancelled_flights_analysis["Cancelled"]
cancelled_flights_analysis["Pct_Cancelled"] = (cancelled_flights_analysis["Cancelled"] / cancelled_flights_analysis["Total"])*100
cancelled_flights_analysis["Airline"] = cancelled_flights_analysis["Operating_Airline"].map(airline_mapper)


In [ ]:
cancelled_flights_analysis.head()     ## TODO: add geographical information of flight, weather information

In [ ]:
cancelled_flights_analysis.set_index("Operating_Airline").sort_values(
    "Cancelled")["Cancelled"].plot(kind="barh", 
                                   figsize=(8, 8), 
                                   width=.8,
                                   edgecolor="black",
                                   title="Flight Cancellations by Airline in 2022")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
# cancelled_flights_analysis["Airline"] = cancelled_flights_analysis["Operating_Airline"].map(
#     airline_mapper)
cancelled_flights_analysis.set_index("Airline").sort_values(
    "Cancelled")["Cancelled"].plot(kind="barh", 
                                   title="Total Operations Cancelled by Airline in 2022",
                                   width=1
                                   )

ax.bar_label(ax.containers[0], padding=5, fontsize=9, color='b')
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
cancelled_flights_analysis["Airline"] = cancelled_flights_analysis["Operating_Airline"].map(
    airline_mapper)
cancelled_flights_analysis.set_index("Airline").sort_values(
    "Pct_Cancelled")["Pct_Cancelled"].plot(kind="barh",
                                           title="Percentage of Operations Cancelled by Airline in 2022",
                                           width=1,
                                           #edgecolor="black",
                                           color=current_palette[6]
                                           )
ax.bar_label(ax.containers[0], fmt='%.3f%%', padding=5, fontsize=9, color='b')
plt.show()

In [ ]:
cancel_count = df_flights_012022_082022.query('20220101 < FlightDate < 20221231').query("Cancelled")["Operating_Airline"].value_counts()
cancel_count

In [ ]:
current_palette

Analysis for January - June 2022

In [ ]:
df_flights_012022_082022["Month"].value_counts()

In [ ]:
df_flights_012022_082022["Quarter"].value_counts()

In [ ]:
df_flights_012022_082022["FlightDate"].value_counts()

In [ ]:
cancelled_flights_analysis = (
    (df_flights_012022_082022.query("20220101 < FlightDate < 20221231")
     .groupby(["Operating_Airline", "Cancelled"])['Cancelled']
     .size()
     .unstack()
     .reset_index()
     )
    .rename(columns={False: "Departed", True: "Cancelled"})
    .rename_axis(None, axis=1))

cancelled_flights_analysis["Total"] = cancelled_flights_analysis["Departed"] + cancelled_flights_analysis["Cancelled"]
cancelled_flights_analysis["Pct_Cancelled"] = (cancelled_flights_analysis["Cancelled"] / cancelled_flights_analysis["Total"])*100
cancelled_flights_analysis["Airline"] = cancelled_flights_analysis["Operating_Airline"].map(airline_mapper)

fig, ax = plt.subplots(figsize=(8, 8))
cancelled_flights_analysis["Airline"] = cancelled_flights_analysis["Operating_Airline"].map(
    airline_mapper)
cancelled_flights_analysis.set_index("Airline").sort_values(
    "Pct_Cancelled")["Pct_Cancelled"].plot(kind="barh",
                                           title="Percentage of Operations Cancelled by Airline in January - June 2022",
                                           width=1,
                                           #edgecolor="black",
                                           color=current_palette[2]
                                           )
ax.bar_label(ax.containers[0], fmt='%.3f%%', padding=5, fontsize=9, color='b')
plt.show()